In [1]:
import re
import numpy as np
import os
from haystack import Document
from haystack.nodes import FARMReader

/home/abhor/miniconda3/envs/materials/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load a reference text

In [9]:
file = "reference_txts/reference_3.txt"
paragraphs = open(file, 'r').readlines()

/tmp/ipykernel_2484151/3210507208.py:2: ResourceWarning: unclosed file <_io.TextIOWrapper name='reference_txts/reference_3.txt' mode='r' encoding='UTF-8'>
  paragraphs = open(file, 'r').readlines()


In [10]:
paragraphs

['1.  Introduction\n',
 'In  general,  structural  materials  that  are  currently  available  for most  of  the  applications  are  based  on  single  principle  elements viz.  Fe-based  steels,  Al-based  alloys,  Ni-based  superalloys,  etc.\n',
 'Although,  solid  solution  strengthening  by  more  than  one  solute  in equi-molar  proportion  is  a  viable  option,  processing  of  alloys  based on  multi-principal  elements  was  not  attempted  for  a  long  time on  the  notion  that  they  may   form  complex  compounds  with  complex  crystal  structures.  However,  the  ﬁrst  impressive  report  by  Yeh et  al.  [1],   followed  by  a  few  research  groups  [2–5],  has  emphatically shown  that  multi-component  high  entropy  alloys  (HEAs)  can  be processed  to  form  simple  solid  solutions  under  appropriate  conditions.  The  term  “high  entropy”  in  HEAs  originates  from  the  resultant high  entropy  of  mixing  (>1.61R,  where  R  is  gas  constant)  when more

Load a QA LLM

In [11]:
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

## First, we find all compositions talked about in the paper.

In [12]:
# Small function to post process answers
def postprocess_answers(ans):
    # Remove extra white-spaces (makes it hard to get exact matches)
    return re.sub("[ ]{2,}", " ", ans)

In [13]:
temp_list = []

for para in paragraphs:
    answers = reader.predict(
        query="Which multicomponent alloy is discussed? For example: AuAgFeCu",
        documents=[Document(content=para)],
        top_k=10
    )
    # print(" || ".join([f"{x.answer}({round(x.score, 2)})" for x in answers["answers"]]))
    if len(answers["answers"]) > 0:
        ans = answers["answers"][0].answer
        ans = postprocess_answers(ans)
        temp_list.append(ans)

print(temp_list)

Inferencing Samples: 100%|████████████████████████████████████████| 1/1 [00:00<00:00, 91.40 Batches/s]

['Introduction', 'Ni-based superalloys', 'high entropy', 'mechanical', 'AlCoCrCuFe', 'NiCoCuFe', 'NiCoCrCuFe', 'AlCoCrCuFe', 'nanocrystalline', 'AlCoCrCuFe', 'Experimental details', 'AlCoCrCuFe', 'NiCoFe HEAs', '3. Results', 'pellets', 'AlCoCrCuFe and NiCoCrCuFe', 'NiCoCrCuFe', 'NiCoFe, NiCoCrFe and NiCoCuFe together with NiCoCrCrFe', 'NiCoCrFe', 'AlCoCrCuFe', 'AlCoCrCuFe', 'NiCoCrCuFe', 'NiCoCuFe, NiCoCrCuFe, AlCoCrCuFe', 'HEAs', 'AlCoCrCuFe', 'NiCoCrFe', '83– 89 Table 1', 'MA', 'Crystallite', 'MA', 'Cu', 'BCC', 'Cu rich FCC', 'sigma', 'Relative density obtained for the SPS samples.', 'Vickers', 'NiCoCrCuFe', 'NiCoCrCuFe', 'AlCoCrCuFe', 'NiCoCrCuFe', 'XRD pattern (which is not clearly visible in the SEM images).', 'AlCoCrCuFe', 'Crystallite size and Vickers hardness', 'AlCoCrCuFe', 'AlCoCrCuFe', 'NiCoCrCuFe', 'NiCoCuFe', 'equi-atomic HEAs', '4.6%),', '83– 89 87', 'NiCoCrCuFe', 'AlCoCrCuFe', 'HEAs.', 'Enthalpy of mixing (kJ/mol) between elements.', 'Smix', 'NiCoFe', 'Cu', 'equi-atomic 

In [14]:
list_of_alloys = []

for alloy in temp_list:
    isalloy = re.search(r"([A-Z][a-z][0-9\.]*){2,}", alloy)
    if isalloy is not None and alloy == isalloy.group(0) and alloy not in list_of_alloys:
        list_of_alloys.append(alloy)

print(list_of_alloys)

['AlCoCrCuFe', 'NiCoCuFe', 'NiCoCrCuFe', 'NiCoCrFe', 'NiCoFe']


## Now, next we want to extract processing condition.

In [15]:
alloy = list_of_alloys[1]
list_of_answers = []
list_of_scores = []

for para in paragraphs:
    answers = reader.predict(
        # query=f"What is the preparation method of {alloy}?",
        query=f"How is {alloy} made?", #+ " among as-cast, additive manufacturing, Bridgman solidification, cold-pressed, mechanically alloyed plus spark plasma sintered, melt-spun, sputter deposition, splat quenched?",
        documents=[Document(content=para)],
        top_k=10
    )
    # print(" || ".join([f"{x.answer}({round(x.score, 2)})" for x in answers["answers"]]))
    if len(answers["answers"]) > 0:
        list_of_answers.extend(list(map(lambda x: postprocess_answers(x.answer), answers["answers"])))
        list_of_scores.extend(list(map(lambda x: x.score, answers["answers"])))

# Remove the answers with "no confidence"
list_of_answers = np.array(list_of_answers)[np.array(list_of_scores) > 0.1]
list_of_answers

Inferencing Samples: 100%|████████████████████████████████████████| 1/1 [00:00<00:00, 91.93 Batches/s]


array(['mechanical alloying', 'equi-atomic alloys',
       'mechanically alloyed', 'HEAs',
       'mechanically alloyed powders of AlCoCrCuFe and NiCoCrCuFe as a function of milling time',
       'milling time', 'sintering', 'Cu', 'deconvoluted', 'phase',
       'lattice parameter', 'MA in metastable form'], dtype='<U333')

In [16]:
dict_of_processing = {
    "AC": ["cast", "AC"],
    "MA": ["mechanical", "MA"],
}

# For testing
# for item in sum(dict_of_processing.values(), []):
#     temp_l = list(filter(lambda x: item in x, list_of_answers))
#     print(item, len(temp_l), temp_l)

In [17]:
# Do some voting....
def get_num_answers_containing_search(answers, list_of_search_items):
    return len(set(sum([list(filter(lambda x: item in x, answers)) for item in list_of_search_items], [])))

def get_highest_voted_item_from_dict_in_answers(answers, dicty):
    new_dict = {}
    for key in dicty:
        new_dict[key] = get_num_answers_containing_search(answers, dicty[key])
    # print(new_dict)
    keys = list(new_dict.keys())
    values = [new_dict[key] for key in new_dict]
    return keys[np.argmax(values)]

get_highest_voted_item_from_dict_in_answers(list_of_answers, dict_of_processing)

'MA'

Now, SS or IM

In [18]:
alloy = list_of_alloys[2]
list_of_answers = []
list_of_scores = []

for para in paragraphs:
    answers = reader.predict(
        # query=f"What is the preparation method of {alloy}?",
        query=f"{alloy} is solid solution, intermetallic or amorphous?", #+ " among as-cast, additive manufacturing, Bridgman solidification, cold-pressed, mechanically alloyed plus spark plasma sintered, melt-spun, sputter deposition, splat quenched?",
        documents=[Document(content=para)],
        top_k=10
    )
    print(" || ".join([f"{x.answer}({round(x.score, 2)})" for x in answers["answers"]]))
    if len(answers["answers"]) > 0:
        list_of_answers.extend(list(map(lambda x: postprocess_answers(x.answer), answers["answers"])))
        list_of_scores.extend(list(map(lambda x: x.score, answers["answers"])))

Inferencing Samples: 100%|████████████████████████████████████████| 1/1 [00:00<00:00, 63.14 Batches/s]


Introduction(0.0)


Inferencing Samples: 100%|████████████████████████████████████████| 1/1 [00:00<00:00, 63.20 Batches/s]


Ni-based  superalloys(0.0)


Inferencing Samples: 100%|████████████████████████████████████████| 1/1 [00:00<00:00, 42.15 Batches/s]


...

solid(0.03)


Inferencing Samples: 100%|████████████████████████████████████████| 1/1 [00:00<00:00, 89.08 Batches/s]


881–893(0.0)


Inferencing Samples: 100%|████████████████████████████████████████| 1/1 [00:00<00:00, 90.68 Batches/s]

1263–1271.(0.0)


In [19]:
list_of_answers = np.array(list_of_answers)[np.array(list_of_scores) > 0.1]
list_of_answers

array(['solid solution', 'solid solution'], dtype='<U547')

In [20]:
# Some voting again... 
dict_of_SS_IM = {
    "SS": ["solid solution", "solid-solution"],
    "IM": ["intermetallic"]
}

get_highest_voted_item_from_dict_in_answers(list_of_answers, dict_of_SS_IM)

'SS'

Phases

In [21]:
alloy = list_of_alloys[1]
list_of_answers = []
list_of_scores = []

for para in paragraphs:
    answers = reader.predict(
        # query=f"What is phase of {alloy}?", 
        query=f"Is {alloy} FCC or BCC?", 
        documents=[Document(content=para)],
        top_k=10
    )
    # print(" || ".join([f"{x.answer}({round(x.score, 2)})" for x in answers["answers"]]))
    if len(answers["answers"]) > 0:
        list_of_answers.extend(list(map(lambda x: postprocess_answers(x.answer), answers["answers"])))
        list_of_scores.extend(list(map(lambda x: x.score, answers["answers"])))

Inferencing Samples: 100%|████████████████████████████████████████| 1/1 [00:00<00:00, 94.52 Batches/s]


In [22]:
list_of_answers = np.array(list_of_answers)[np.array(list_of_scores) > 0.1]
list_of_answers

array(['NiCoCuFe', 'BCC', 'BCC', 'FCC', 'FCC', 'FCu', 'BCC', 'BCC', 'BCC',
       'FCC', 'BCC', 'FCC', 'FCC', 'BCC', 'BCC', 'BCC', 'BCC', 'BCC'],
      dtype='<U270')

In [23]:
# Some voting again... 
dict_of_phases = {
    "BCC": ["BCC"],
    "FCC": ["FCC"]
}

get_highest_voted_item_from_dict_in_answers(list_of_answers, dict_of_SS_IM)

'SS'

## Property explored in paper

In [24]:
list_of_answers = []
list_of_scores = []

for para in paragraphs:
    answers = reader.predict(
        query="What is measured?", # Alloy agnostic
        documents=[Document(content=para)],
        top_k=10
    )
    # print(" || ".join([f"{x.answer}({round(x.score, 2)})" for x in answers["answers"]]))
    if len(answers["answers"]) > 0:
        list_of_answers.extend(list(map(lambda x: postprocess_answers(x.answer), answers["answers"])))
        list_of_scores.extend(list(map(lambda x: x.score, answers["answers"])))

list_of_answers = np.array(list_of_answers)[np.array(list_of_scores) > 0.1]
list_of_answers

Inferencing Samples: 100%|████████████████████████████████████████| 1/1 [00:00<00:00, 92.17 Batches/s]


array(['high entropy', '99.5%', 'Hardness', 'hardness', 'XRD patterns',
       'XRD', 'milling time', 'XRD peaks', 'FCC phases', 'segregation',
       'de-convoluted peaks',
       'XRD patterns of AlCoCrCuFe and NiCoCrCuFe after SPS. Inset shows the deconvoluted form of XRD peaks',
       'de-convoluted form of XRD peaks', 'Crystallite size',
       'sigma phase', 'density', 'relative density', 'Spot EDS analysis',
       'experimental density', 'Crystallite size and lattice strain',
       'Williamson–Hall analysis', 'hardness', 'entropy',
       'parameters are calculated for the alloy systems',
       'BSE-SEM images of SPS pellets', 'kJ/mol', 'enthalpy',
       'sigma phase', 'entropy', 'a shift in the Ni peaks',
       '3.52 and 3.61 ˚A', '3.58, 3.60, 3.59 and 3.62 ˚A', 'density'],
      dtype='<U318')

In [25]:
# Some voting again... 
dict_of_properties = {
    "H": ["hardness", "Hardness"],
    "M": ["Mechanical", "microstructure", "strength", "compress"]
}

get_highest_voted_item_from_dict_in_answers(list_of_answers, dict_of_SS_IM)

'SS'

# LET'S MAKE THAT TABLE NOW!

In [26]:
import re

def get_paragraphs(file):
    return open(file, 'r').readlines()

# Small function to post process answers
def postprocess_answers(ans):
    # Remove extra white-spaces (makes it hard to get exact matches)
    return re.sub("[ ]{2,}", " ", ans)

def run_query_on_paragraphs(model, paragraphs, query, threshold=0.1):
    '''
    model: QA model
    paragraphs: list of strings to run query on
    query: string to use for QA model's query
    threshold: Answers with confidence above this threshold will be returned
    '''
    list_of_answers = []
    list_of_scores = []

    for para in paragraphs:
        answers = model.predict(
            query=query,
            documents=[Document(content=para)],
            top_k=10
        )
        # print(" || ".join([f"{x.answer}({round(x.score, 2)})" for x in answers["answers"]]))
        if len(answers["answers"]) > 0:
            list_of_answers.extend(list(map(lambda x: x.answer, answers["answers"])))
            list_of_scores.extend(list(map(lambda x: x.score, answers["answers"])))
    
    # Remove the answers with "no confidence"
    list_of_answers = np.array(list_of_answers)[np.array(list_of_scores) > 0.1]
    return list_of_answers

def filter_alloy_compositions(l):
    '''
    takes a list of string and returns those that have an alloy composition
    '''
    list_of_alloys = []

    for alloy in l:
        isalloy = re.search(r"([A-Z][a-z][0-9\.x]*){2,}", alloy)
        if isalloy is not None and alloy == isalloy.group(0) and alloy not in list_of_alloys:
            list_of_alloys.append(alloy)
    
    return list_of_alloys

In [27]:
import glob
import pandas as pd
from haystack.nodes import FARMReader

%time reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

CPU times: user 3.07 s, sys: 451 ms, total: 3.52 s
Wall time: 3.01 s


In [28]:
results = pd.DataFrame(columns = ["Composition", "Processing Condition", "SS or IM", "Phases", "Property", "References"])

for file in glob.glob("reference_txts/*"):
    # print(file)
    ref = int(re.findall("reference_([0-9]+).txt", file)[0])
    
    # Get content from reference
    paragraphs = get_paragraphs(file) 
    # print("Paragraphs:", len(paragraphs))

    # Get alloys
    answers = run_query_on_paragraphs(reader, paragraphs,
        query="Which multicomponent alloy is discussed? For example: AuAgFeCu", threshold=0)
    list_of_alloys = filter_alloy_compositions(answers)
    # assert len(list_of_alloys) > 0, f"File = {file}"

    # Get the property (global)
    answers = run_query_on_paragraphs(reader, paragraphs, "What is measured?")
    pr = get_highest_voted_item_from_dict_in_answers(answers, dict_of_properties)
    
    # For each alloy, get other details
    for alloy in list_of_alloys:
        # preprocessing condition
        answers = run_query_on_paragraphs(reader, paragraphs, f"How is {alloy} made?")
        pc = get_highest_voted_item_from_dict_in_answers(answers, dict_of_processing)

        # SS or IM
        answers = run_query_on_paragraphs(reader, paragraphs, f"{alloy} is solid solution, intermetallic or amorphous?")
        si = get_highest_voted_item_from_dict_in_answers(answers, dict_of_SS_IM)

        # Phases
        answers = run_query_on_paragraphs(reader, paragraphs, f"Is {alloy} FCC or BCC?")
        ph = get_highest_voted_item_from_dict_in_answers(answers, dict_of_phases)

        results.loc[results.shape[0] + 1] = [alloy, pc, si, ph, pr, ref]
    # break

/tmp/ipykernel_2484151/3534423196.py:4: ResourceWarning: unclosed file <_io.TextIOWrapper name='reference_txts/reference_6.txt' mode='r' encoding='UTF-8'>
  return open(file, 'r').readlines()


reference_txts/reference_6.txt
Paragraphs: 34


Inferencing Samples: 100%|████████████████████████████████████████| 1/1 [00:00<00:00, 92.13 Batches/s]


37 37


Inferencing Samples: 100%|████████████████████████████████████████| 1/1 [00:00<00:00, 93.96 Batches/s]


...

52 52


Inferencing Samples: 100%|████████████████████████████████████████| 1/1 [00:00<00:00, 61.34 Batches/s]


49 49


Inferencing Samples: 100%|████████████████████████████████████████| 1/1 [00:00<00:00, 89.89 Batches/s]

48 48


In [29]:
results

,Composition,Processing Condition,SS or IM,Phases,Property,References
1,AlCoCrCuFeNiMox,AC,SS,BCC,M,6
2,LaNi5,AC,SS,BCC,H,101
3,AlCoCrCuFeNi,AC,SS,BCC,H,102
4,Ta20Nb20Hf20Zr20Ti20,AC,SS,BCC,M,112
5,FeNiCr,AC,SS,FCC,H,89
6,Fe20Cr20Mn20Ni20Co20,AC,SS,FCC,M,86
7,AlCrFeCoNi,AC,SS,BCC,M,53
8,AlxCoCrFeNi,AC,SS,BCC,H,56
9,AlCoCrCuFeNi,AC,SS,BCC,H,28
10,CoCrFeNiMox,AC,SS,BCC,H,93


In [36]:
results.to_csv("results/basic_QA.csv")

Remember to track memory, especially GPU! Shut this notebook before running another!